// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0

# Fine-Tune Amazon Nova model provided by Amazon Bedrock: End-to-End

This notebook demonstrates the end-to-end process of fine-tuning Amazon Nova Lite and Amazon Nova Micro model using Amazon Bedrock, including selecting the base model, configuring hyperparameters, creating and monitoring the fine-tuning job, deploying the fine-tuned model with provisioned throughput and evaluating the performance of the fine-tuned model. 

Note: The following steps can also be done through the Amazon Bedrock Console

# Prerequisites

- Make sure you have prepared a fine-tuning dataset following the format required [here]( 
https://docs.aws.amazon.com/nova/latest/userguide/customize-fine-tune-prepare.html)
- Make sure your AWS account has appropriate permissions (e.g. access to Amazon Bedrock (us-east-1))

In [ ]:
!pip install -qU -r requirements.txt

In [ ]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# Setup

In [3]:
import boto3 
from botocore.config import Config
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json
import time 
import concurrent.futures
import shortuuid
import tqdm
import os

In [4]:
my_config = Config(
    region_name = 'us-east-1', 
    signature_version = 'v4',
    retries = {
        'max_attempts': 5,
        'mode': 'standard'
    })

bedrock = boto3.client(service_name="bedrock", config=my_config)

In [ ]:
## Specify input S3 bucket

input_s3_uri = "s3://ft-aws-domain/ft_bedrock_data/ft_olympus_jsonl/aws_train_olympus.jsonl"
output_s3_uri = "s3://ft-aws-domain/model_output/ft_nova_lite_aws_v1/"

# Select the base model to fine-tune

You need to provide the `base_model_id` for the model you want to fine-tune. You can find a list of the foundational model ids by invoking the `list_foundation_models` API:

``` \n
for model in bedrock.list_foundation_models(
    byCustomizationType="FINE_TUNING")["modelSummaries"]:
    for key, value in model.items():
        print(key, ":", value)
    print("-----\n")
```

In [24]:
nova_micro_identifier = "arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0:128k"
nova_lite_identifier = "arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0:300k"

Next, provide the `customization_job_name`, `custom_model_name` and `customization_role` which will be used to create the fine-tuning job.

In [ ]:
# Nova model customization currently only available in US EAST 1

role_name = # Replace with your role name
role_arn = # Replace with your role ARN

job_name = "aws-ft-nova-lite-v1"
model_name = job_name 

# Create fine-tuning job

<div class=\"alert alert-block alert-info\">
    <b>Note:</b> Fine-tuning job will take around 2-4 hrs to complete.</div>

| ***Parameter Name*** | ***Parameter Description*** | ***Type*** | ***Min*** | ***Max*** | **Default** |
| ------- | ------------- | ------ | --------- | ----------- | ----------- |
| Epochs | The maximum number of iterations through the entire training dataset | integer | 1 | 5 | 2 |
| Learning rate | The rate at which model parameters are updated after each batch of training data | float | 1.00E-06 | 1.00E-04 | 1.00E-05 |
| Learning rate warmup steps | Number of iterations over which learning rate is gradually increased to the initial rate specified | integer | 0 | 20 | 10 |
| Batch size | The number of samples processed before updating model parameters | integer | NA | NA | Fixed at 1|

In [25]:
# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
customization_type = "FINE_TUNING"


# Define the hyperparameters for fine-tuning Amazon Nova model
hyper_parameters = {
        "epochCount": "1",
        "learningRate": '0.000001', 
        "batchSize": "1",
    }


response_ft = bedrock.create_model_customization_job(
    customizationType=customization_type,
    jobName = job_name,
    customModelName = model_name,
    roleArn = role_arn,
    baseModelIdentifier = nova_lite_identifier,
    hyperParameters=hyper_parameters,
    trainingDataConfig={"s3Uri": input_s3_uri},
    outputDataConfig={"s3Uri": output_s3_uri},
)

# Check fine-tuning job status

In [26]:
jobArn = response_ft.get('jobArn')
status = bedrock.get_model_customization_job(jobIdentifier=jobArn)["status"]
print(f'Job status: {status}')

Job status: InProgress


# Setup provisioned throughput

Once the job status changes to `complete`, we need to create provisioned throughput which is needed for running inference on the fine-tuned Amazon Nova model. For more information on provisioned throughput, please refer to [this documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/prov-throughput.html)

In [ ]:
provisioned_model_name = 'finetuned_nova_lite'
custom_model_id = 'aws-ft-nova-lite-v1'

provisioned_model_id = bedrock.create_provisioned_model_throughput(
                                        modelUnits=1,
                                        provisionedModelName=provisioned_model_name,
                                        modelId=custom_model_id
                            )

print(provisioned_model_id['provisionedModelArn'])

In [ ]:
status_provisioning = bedrock.get_provisioned_model_throughput(provisionedModelId = provisioned_model_id)['status']

import time
while status_provisioning == 'Creating':
    time.sleep(60)
    status_provisioning = bedrock.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id)['status']
    print(status_provisioning)
    time.sleep(60)

# Delete provisioned throughput

<b>Warning</b>: Please make sure to delete providsioned throughput as there will cost incurred if its left in running state, even if you are not using it.

In [ ]:
bedrock.delete_provisioned_model_throughput(provisionedModelId=provisioned_model_id)